# Irrigation Field Identification and Analysis (Step 2)

This notebook implements the **second step** in the field-level analysis workflow: identifying irrigated agricultural fields and quantifying irrigation patterns based on the ET data prepared in the previous step.

## Processing Logic:

1. **Threshold-Based Classification**: Applies ETa/ETc and ET blue thresholds to identify potentially irrigated fields
   - ETa/ETc threshold (default: 0.7) to detect water stress relief
   - ET blue threshold (default: 3.0 mm) to identify significant irrigation amounts

2. **Temporal Filtering**: Processes irrigation data across the growing season (May-September)
   - Filters out winter crops and double-cropping scenarios
   - Focuses on main irrigation period for Swiss agriculture

3. **Quality Control**: Removes unreliable data based on field characteristics
   - Minimum field size threshold (5000 m²)
   - Zero fraction filtering to exclude fields with excessive missing data
   - Spatial coherence checks

4. **Irrigation Quantification**: Calculates field-level irrigation metrics
   - Monthly ET blue aggregation for seasonal patterns
   - Classification into irrigation intensity classes (1-5)
   - Statistical analysis by crop type and temporal patterns

5. **Export and Validation**: Outputs processed irrigation data for further analysis
   - Time series of irrigation per field
   - Summary statistics by class and temporal period
   - Quality assessment and validation metrics

## Workflow Position:
- **Previous Script**: Field-level ET data preparation (`field_level_postprocessing.ipynb`)
- **Current Script**: Irrigation field identification and analysis
- **Output**: Classified irrigation time series ready for regional aggregation

This script transforms the preprocessed ET field data into actionable irrigation intelligence, identifying which fields are irrigated, when irrigation occurs, and quantifying irrigation amounts across different agricultural systems.

In [487]:
import ee
import sys
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Union
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geemap

# Add the project root to the path so we can import our modules
sys.path.append('..')

# Initialize Earth Engine
# ee.Authenticate()
ee.Initialize(project="thurgau-irrigation")
# ee.Initialize(project="ee-sahellakes")

print("✓ Libraries imported and Earth Engine initialized successfully")

✓ Libraries imported and Earth Engine initialized successfully


In [488]:
# Import our refactored field-level processing class
from src.et_blue_per_field.field_level_postprocessing import FieldLevelETProcessor, ProcessingConfig

# Import utility functions
from utils.ee_utils import back_to_float
from utils.date_utils import merge_same_date_images

from src.et_green.filter_nutzungsflaechen import get_winter_crops

print("✓ Field-level ET processing modules imported successfully")

✓ Field-level ET processing modules imported successfully


In [489]:
# Define the canton to process (single point of configuration)
CANTON = "Zuerich"  # Can be changed to "Thurgau", "Zuerich", etc.
# Define parameters - using the globally configured canton
year = 2018
MINIMUM_FIELD_SIZE = 5000  # minimum field size in m^2
print(f"✓ Canton configured: {CANTON}")


✓ Canton configured: Zuerich


In [490]:
# Configuration for processing
YEAR = year

# output_asset_path = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field/ETblue_field_ts_{CANTON}_{YEAR}_Bibertal"
# ETa_ETc threshold
ETa_ETc_threshold = 0.7 #0.65
# ETblue threshold
ETblue_threshold = 10 #12

output_asset_path = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_{CANTON}_{YEAR}"

##add parameter values (ETa_ETc threshold, ETblue threshold) to output path
output_asset_path = output_asset_path + f"_ETaETc{int(ETa_ETc_threshold*100)}_ETblue{ETblue_threshold}"
output_asset_path_stats =f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_annual_v2_withETA/ETblue_field_stats_{CANTON}_{YEAR}" + f"_ETaETc{int(ETa_ETc_threshold*100)}_ETblue{ETblue_threshold}"

# Create processing configuration
config = ProcessingConfig(
    year=YEAR,
    cantons=["Schaffhausen","Thurgau", "Zuerich"],  # Same as original JS
    base_et_path="projects/thurgau-irrigation/assets/{canton}/ET_products/decadal_Landsat_30m",
    etf_base_path="projects/thurgau-irrigation/assets/{canton}/ET_products/decadal_Landsat_30m_ETF",
    etf_modeled_path="projects/thurgau-irrigation/assets/ZH_SH_TG/ETF/ETF_Weiden_dekadal_from_Landsat_30m_v3",
    base_et_green_path="projects/thurgau-irrigation/assets/ZH_SH_TG/ET_green",
    landuse_path="projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_{year}_Kulturen_with_veg_period",
    vegetation_period_path="projects/thurgau-irrigation/assets/ZH_SH_TG/crop_vegetation_period_{year}_harmonic",
    wald_proximity_path="projects/thurgau-irrigation/assets/Wald_SWISSTLM3D_2023_proximity",
    landuse_property_name="nutzung"
)

print(f"✓ Configuration set up for year {YEAR}")
print(f"  - Processing cantons: {config.cantons}")
print(f"  - Landuse property: {config.landuse_property_name}")
print(f"  - Output asset path: {output_asset_path}")


✓ Configuration set up for year 2018
  - Processing cantons: ['Schaffhausen', 'Thurgau', 'Zuerich']
  - Landuse property: nutzung
  - Output asset path: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Zuerich_2018_ETaETc70_ETblue10


In [492]:
# Load the landuse collection for the configured canton
landuse_path = config.landuse_path.format(year=year)
print(f"Loading landuse data from: {landuse_path}")
landuse_collection = ee.FeatureCollection(landuse_path)
# Make unique ID
landuse_collection = landuse_collection.map(lambda f: f.set('field_id', f.id()))

# Define the input asset path
input_asset_path = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field/ETblue_field_ts_{CANTON}_{YEAR}"
# read in the input asset
et_blue_collection = ee.FeatureCollection(input_asset_path)
#print out the size of the collection
print(f"ET blue collection size: {et_blue_collection.size().getInfo()}")

geometry = ee.Geometry.Point([8.827134274374572, 47.68503138489175])
print(landuse_collection.filterBounds(geometry).first().get('field_id').getInfo())



Loading landuse data from: projects/thurgau-irrigation/assets/ZH_SH_TG/Nutzungsflaechen/ZH_SH_TG_2018_Kulturen_with_veg_period
ET blue collection size: 153390
000000000000000065fb


In [493]:
import pandas as pd

# Get the table data
table = et_blue_collection.filter(ee.Filter.eq('field_id', '000000000000000009c0')).sort('date').getInfo()

# Extract features from the table
features = table['features']

# Convert to DataFrame
if features:
    # Extract properties from each feature
    data = []
    for feature in features:
        data.append(feature['properties'])
    
    df = pd.DataFrame(data)
    
    # Display the table
    print("=" * 100)
    print(f"FIELD DATA FOR FIELD ID")
    print("=" * 100)
    print(df.to_string(index=False))
    
    # Optional: Show specific columns if the table is too wide
    key_columns = ['date', 'month', 'decade', 'ET_blue', 'ETa_ETc', 'class', 'nutzung', 'monthly_ET_blue']
    available_columns = [col for col in key_columns if col in df.columns]
    
    if len(df.columns) > 10:  # If too many columns, show key ones
        print(f"\nKEY COLUMNS ONLY:")
        print("=" * 60)
        print(df[available_columns].to_string(index=False))
else:
    print("No data found for field ID")

No data found for field ID


In [ ]:
# Get winter crops
winter_crops = get_winter_crops()#alternatively use class equal to 5
winter_crops = ee.List(list(winter_crops))  # Convert set to list before creating ee.List
# Filter out winter crops that are not double-cropped
et_blue_collection = et_blue_collection.filter(ee.Filter.And(ee.Filter.eq('isDoubleCropping', 0),ee.Filter.inList('nutzung',winter_crops)).Not())
# #give me the distribution of the property 'zero_fraction' in my collection
# zero_fraction_list = et_blue_collection.aggregate_array('zero_fraction').reduce(ee.Reducer.frequencyHistogram())
# print("Distribution of 'zero_fraction':", zero_fraction_list.getInfo())
et_blue_collection = et_blue_collection.filter(ee.Filter.Or(ee.Filter.lt('zero_fraction', 0.4),ee.Filter.gte('zero_fraction', 1)))

#print out the size of the collection
print(f"ET blue collection size before filtering: {et_blue_collection.size().getInfo()}")
unique_field_ids = et_blue_collection.aggregate_array('field_id').distinct().size()
# print(f"Number of unique fields: {unique_field_ids.getInfo()}")
bin_distribution = et_blue_collection.aggregate_array('class').reduce(ee.Reducer.frequencyHistogram())
# print("Distribution of Classes:", bin_distribution.getInfo())
# Get all unique classes
unique_classes = et_blue_collection.aggregate_array('class').distinct().sort()
#number of unique fields per class
field_counts_per_class = unique_classes.map(lambda class_num: 
    et_blue_collection.filter(ee.Filter.eq('class', class_num))
    .aggregate_array('field_id').distinct().size())
# print(f"Number of unique fields per class: {dict(zip(unique_classes.getInfo(), field_counts_per_class.getInfo()))}")
# Fraction of dates with data available
fraction_dates_available = ee.Number(et_blue_collection.aggregate_array('ET_blue').reduce(ee.Reducer.count())).divide(
        ee.Number(et_blue_collection.size())
)
# print(f"Fraction of dates with data available: {fraction_dates_available.getInfo()}")
fractions_per_month = ee.List.sequence(5,9).map(lambda m: 
    ee.Number(et_blue_collection.filter(ee.Filter.eq('month',m)).aggregate_array('ET_blue').reduce(ee.Reducer.count())).divide(
        ee.Number(et_blue_collection.filter(ee.Filter.eq('month',m)).size())
    )
)
# # Fraction of dates with data available
# print(f"Fractions of dates available per month: {dict(zip(ee.List.sequence(5,9).getInfo(), fractions_per_month.getInfo()))}")

#kick out decades without data
et_blue_collection = et_blue_collection.filter(ee.Filter.neq('zero_fraction', 1))
et_blue_collection = et_blue_collection.filter(ee.Filter.gte('ETF_modeled', 0))

# In the Landsat Collection 2 dataset, ETf = ETa/ET0 for reference grass, available as part of the dataset,
# is divided by Kc to obtain ETc without additional computation. We have exported ETf, now we must divide by Kc to get ETa/ETc.
kc_dict = ee.FeatureCollection('projects/thurgau-irrigation/assets/FribourgAndVaud/ETc_WAPOR/Kc_Pasture_Broye_Aquastat')
def get_ETa_ETc(feature):
    month = ee.Number(feature.get('month'))
    decade = ee.Number(feature.get('decade'))
    # Get the Kc value for the month
    kc_feature = kc_dict.filter(ee.Filter.And(ee.Filter.eq('Month', month), ee.Filter.eq('Decade', decade))).first()
    kc_value = ee.Number(kc_feature.get('Kc')) 
    # Divide ET_blue by Kc to get ETa/ETc
    ETF = ee.Number(feature.get('ETF_modeled'))
    ETa_ETc = ETF.divide(kc_value)
    return feature.set('ETa_ETc', ETa_ETc)

et_blue_collection = et_blue_collection.map(get_ETa_ETc)

# # Property names
# print("Property names after filtering:", et_blue_collection.filter(ee.Filter.gt('ET_blue',0)).first().propertyNames().getInfo())
# Filter out winter crops where 'month' is smaller than 'secondStart' or larger than 'secondEnd'.
# Map over the collection and mark features with 'unirrigated' == 1
def mark_unirrigated(feature):
    month = ee.Number(feature.get('month'))
    second_start = ee.Number(feature.get('secondStart'))
    second_end = ee.Number(feature.get('secondEnd'))
    condition = month.lt(second_start).Or(month.gt(second_end))
    # Set 'unirrigated' to 1 if condition is true, otherwise keep original value
    return feature.set('unirrigated', ee.Algorithms.If(condition.And(ee.Number(feature.get('class')).eq(5)), 1, 0))

et_blue_collection = et_blue_collection.map(mark_unirrigated)
et_blue_collection = et_blue_collection.filter(ee.Filter.neq('unirrigated', 1))

# print(f"ET blue collection size after filtering: {et_blue_collection.size().getInfo()}")
# print(f"ET blue collection mad_res size : {et_blue_collection.filter(ee.Filter.gt('mad_res', 0)).size().getInfo()}")
# print(f"ET blue collection mad_res size : {et_blue_collection.aggregate_array('mad_res').length().getInfo()}")
# #print first feature with mad_res equal to null
# first_null_mad_res = et_blue_collection.filter(ee.Filter.eq('mad_res', 0)).first()
# print('First feature with mad_res equal to 0:', first_null_mad_res.getInfo())

et_blue_collection = et_blue_collection.filter(ee.Filter.gt('mad_res', 0))


# Mark features where ETblue exceeds the significance threshold
def mark_irrigated(feature):
    ET_blue = ee.Number(feature.get('ET_blue'))
    # set ET_blue to 0 if it's not defined
    ET_blue = ee.Number(ee.Algorithms.If(ET_blue, ET_blue, 0))
    median_abs_res = ee.Number(feature.get('median_abs_res'))
    mean_abs_res = ee.Number(feature.get('mean_abs_res'))
    stddev_abs_res = ee.Number(feature.get('stddev_abs_res'))
    mad_res = ee.Number(feature.get('mad_res'))

    # Default condition (example: 90% coverage ~ median + 1.28·MAD)
    condition = ET_blue.gt(median_abs_res.add(mad_res.multiply(1.28)))

    # Define z-thresholds as multipliers for MAD (from earlier table)
    # These values correspond to two-sided coverage with median+MAD
    mad_thresholds = {
        'k1':    1.0,     # ~50% coverage
        'k148':  1.4826,  # ~68% (≈1σ equivalent)
        'k243':  2.4387,  # ~90%
        'k291':  2.9058,  # ~95%
        'k382':  3.8189,  # ~99%
        'k416':  4.1617,  # ~99.5%
        'k488':  4.8785   # ~99.9%
    }

    return feature.set('irrigated', ee.Algorithms.If(condition, 1, 0)).set({
        # 'ET_blue_bin': et_blue_bin,
        'residual': median_abs_res.add(mad_res.multiply(2.4387)),  # example: 90% cutoff
        'residual2': mean_abs_res.add(stddev_abs_res.multiply(1.28))  # example: 90% cutoff
    })


et_blue_collection = et_blue_collection.map(mark_irrigated)
# a function to compute the monthly sum of ET_blue
def compute_monthly_et_blue(feature):
    month = ee.Number(feature.get('month'))
    field_id = ee.String(feature.get('field_id'))
    residual = ee.Number(feature.get('residual'))
    ET_blue_sum = et_blue_collection.filter(ee.Filter.lt('ETa_ETc', ETa_ETc_threshold)).filter(ee.Filter.gt('ET_blue', residual)).filter(ee.Filter.eq('month', month)).filter(ee.Filter.eq('field_id', field_id)).aggregate_sum('ET_blue')
    #alternatively, consider also negative ET_blue values? - no, because missing months would then be counted as zero
    return feature.set('monthly_ET_blue', ET_blue_sum)

# Apply the function to each feature in the collection
et_blue_collection = et_blue_collection.map(compute_monthly_et_blue)

#Function to check if the month with significant ET_blue falls within the cropping season
def get_month_of_irrigation(feature):
    month = ee.Number(feature.get('month'))
    monthly_ET_blue = ee.Number(feature.get('monthly_ET_blue'))
    # first_month = ee.Number(feature.get('first_month'))
    # last_month = ee.Number(feature.get('last_month'))
    # Check if this is the first or the second cropping season
    second_start = ee.Number(feature.get('secondStart'))
    second_end = ee.Number(feature.get('secondEnd'))
    first_start = ee.Number(feature.get('firstStart'))
    first_end = ee.Number(feature.get('firstEnd'))
    #Except mais, which is not irrigated in the month of harvest
    first_end = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(2), first_end.subtract(1), first_end))
    second_end = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(2), second_end.subtract(1), second_end))
    #Except Zuckerruebe, which is not irrigated in the first month of the culativation period
    first_start = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(4), first_start.add(1), first_start))
    second_start = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(4), second_start.add(1), second_start))
    
    in_first_season = month.gte(first_start).And(month.lte(first_end))
    in_second_season = month.gte(second_start).And(month.lte(second_end))
    same_season = in_first_season.Or(in_second_season)
    monthly_ET_blue = ee.Number(ee.Algorithms.If(same_season, monthly_ET_blue, 0))
    #round to 1 decimal place
    monthly_ET_blue = monthly_ET_blue.multiply(10).round().divide(10)

    return feature.set('monthly_ET_blue', monthly_ET_blue)

et_blue_collection = et_blue_collection.map(get_month_of_irrigation)


ET blue collection size before filtering: 139040


In [495]:
et_blue_collection=ee.FeatureCollection(output_asset_path)

#map of the collection and round monthly_ET_blue to 1 decimal place
def round_monthly_et_blue(feature):
    monthly_ET_blue = ee.Number(feature.get('monthly_ET_blue'))
    monthly_ET_blue = monthly_ET_blue.multiply(10).round().divide(10)
    return feature.set('monthly_ET_blue', monthly_ET_blue)
et_blue_collection = et_blue_collection.map(round_monthly_et_blue)

# Function to get the first and last month with significant ET_blue per field
def get_month_of_irrigation2(feature):
    field_id = ee.String(feature.get('field_id'))
    # First month and last month with monthly_ET_blue >= ETblue_threshold
    first_month = ee.Number(et_blue_collection.filter(ee.Filter.eq('field_id', field_id)).filter(ee.Filter.gte('monthly_ET_blue', ETblue_threshold)).aggregate_min('month'))
    last_month = ee.Number(et_blue_collection.filter(ee.Filter.eq('field_id', field_id)).filter(ee.Filter.gte('monthly_ET_blue', ETblue_threshold)).aggregate_max('month'))
    # Handle null values by providing defaults
    first_month = ee.Number(ee.Algorithms.If(first_month, first_month, -1))  # Use -1 or another sentinel value
    last_month = ee.Number(ee.Algorithms.If(last_month, last_month, -1))
    return feature.set('first_month', first_month).set('last_month', last_month)

et_blue_collection = et_blue_collection.map(get_month_of_irrigation2)

# now mark again unirrigated fields:
# conditions for irrigated:
# - ETa_ETc > ETa_ETc_threshold
# - monthly_ET_blue > ETblue_threshold for any month for a given field
# - irrigated == 1 for any decade for a given field
# - same cropping season as month of monthly_ET_blue >= ETblue_threshold

#function to mark unirrigated fields
#check for each decade if the field is irrigated in the same cropping season as the month with significant ET_blue
def mark_final_unirrigated(feature):
    field_id = ee.String(feature.get('field_id'))
    month = ee.Number(feature.get('month'))
    ETa_ETc = ee.Number(feature.get('ETa_ETc'))
    monthly_ET_blue = ee.Number(feature.get('monthly_ET_blue'))
    ET_blue = ee.Number(feature.get('ET_blue'))
    irrigated_flag = ee.Number(feature.get('irrigated'))
    # # First month and last month with monthly_ET_blue > ETblue_threshold
    # first_month = ee.Number(et_blue_collection.filter(ee.Filter.eq('field_id', field_id)).filter(ee.Filter.gt('monthly_ET_blue', ETblue_threshold)).aggregate_min('month'))
    # last_month = ee.Number(et_blue_collection.filter(ee.Filter.eq('field_id', field_id)).filter(ee.Filter.gt('monthly_ET_blue', ETblue_threshold)).aggregate_max('month'))
    # # Handle null values by providing defaults
    # first_month = ee.Number(ee.Algorithms.If(first_month, first_month, -1))  # Use -1 or another sentinel value
    # last_month = ee.Number(ee.Algorithms.If(last_month, last_month, -1))
    first_month = ee.Number(feature.get('first_month'))
    last_month = ee.Number(feature.get('last_month'))
    # Check if this is the first or the second cropping season
    second_start = ee.Number(feature.get('secondStart'))
    second_end = ee.Number(feature.get('secondEnd'))
    first_start = ee.Number(feature.get('firstStart'))
    first_end = ee.Number(feature.get('firstEnd'))
    #Except mais, which is not irrigated in the month of harvest
    first_end = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(2), first_end.subtract(1), first_end))
    second_end = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(2), second_end.subtract(1), second_end))
    #Except Zuckerruebe, which is not irrigated in the first month of the culativation period
    first_start = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(4), first_start.add(1), first_start))
    second_start = ee.Number(ee.Algorithms.If(ee.Number(feature.get('class')).eq(4), second_start.add(1), second_start))

    in_first_season = month.gte(first_start).And(month.lte(first_end))
    in_second_season = month.gte(second_start).And(month.lte(second_end))
    # Check if the first and last month with significant ET_blue are in the same season as this month
    first_in_same_season = ee.Number(ee.Algorithms.If(in_first_season, first_month.gte(first_start).And(first_month.lte(first_end)),
                                           ee.Algorithms.If(in_second_season, first_month.gte(second_start).And(first_month.lte(second_end)), 0)))
    last_in_same_season = ee.Number(ee.Algorithms.If(in_first_season, last_month.gte(first_start).And(last_month.lte(first_end)),
                                          ee.Algorithms.If(in_second_season, last_month.gte(second_start).And(last_month.lte(second_end)), 0)))
    same_season = first_in_same_season.Or(last_in_same_season)

    # # Check if any decade for the same field has irrigated == 1
    # any_irrigated = ee.Number(et_blue_collection.filter(ee.Filter.eq('field_id', field_id)).sort('irrigated',False).aggregate_array('irrigated').get(0))
    # Check if any month for the same field has monthly_ET_blue > ETblue_threshold
    any_monthly_et_blue = et_blue_collection.filter(ee.Filter.And(ee.Filter.lt('ETa_ETc', ETa_ETc_threshold), ee.Filter.eq('field_id', field_id))).sort('monthly_ET_blue', False).aggregate_array('monthly_ET_blue')
    any_monthly_et_blue = ee.Number(ee.Algorithms.If(ee.List(any_monthly_et_blue).length().gt(0), ee.List(any_monthly_et_blue).get(0), 0)).gte(ETblue_threshold)
    
    # Final condition for being marked as irrigated
    final_condition = ET_blue.gt(0).And(ETa_ETc.lt(ETa_ETc_threshold)).And(any_monthly_et_blue).And(same_season)#.And(any_irrigated.eq(1))

    return feature.set('unirrigated', ee.Algorithms.If(final_condition, 0, 1))

et_blue_collection = et_blue_collection.map(mark_final_unirrigated)
# print(f"ET blue collection size before filtering: {et_blue_collection.size().getInfo()}")
# print(f"ET blue missing: {et_blue_collection.size().subtract(et_blue_collection.aggregate_array('ET_blue').length()).getInfo()}")
et_blue_collection = et_blue_collection.filter(ee.Filter.gt('ET_blue', -9999))

et_blue_collection2export = et_blue_collection.set('ETa_ETc_threshold', ETa_ETc_threshold).set('ETblue_threshold', ETblue_threshold)

# export_task = ee.batch.Export.table.toAsset(
#     collection=et_blue_collection2export,
#     description=f"ETblue_field_timeseries_{CANTON}_{YEAR}",
#     assetId=output_asset_path
# )
# export_task.start()
# print(f"Export task started: {output_asset_path}")

In [ ]:
###Add ET information to irrigated fields###

ET_BAND_NAME = "ET"
SCALING_FACTOR = 100 ## relevant only for the ETF images, where DYNAMIC is False
DYNAMIC = True
SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
NUMBER_OF_IMAGES = 21
EXPORT_IMAGE_RESOLUTION = 30  # in meters
ET_BAND_RESOLUTION = 30  # 30# in meters

KANTON = "Thurgau"
PATH_TO_ET_PRODUCT_1 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Zuerich"
PATH_TO_ET_PRODUCT_2 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"
KANTON = "Schaffhausen"
PATH_TO_ET_PRODUCT_3 = f"projects/thurgau-irrigation/assets/{KANTON}/ET_products/decadal_Landsat_30m"

et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT_1)
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_2))
    .merge(ee.ImageCollection(PATH_TO_ET_PRODUCT_3))
    .filterDate(f"{YEAR}-05-01", f"{YEAR}-09-30")
)

# Mosaic images with identical dates
et_collection = merge_same_date_images(et_collection).sort("system:time_start")

# Function to calculate median monthly ET for each field_id
def calculate_stats_ET(ft):
    field_id = ee.String(ee.Feature(ft).get('field_id'))
    class_collection = et_collection

    #for loop over months 5 to 9
    for month in range(5, 10):
        month_collection = class_collection.filter(ee.Filter.calendarRange(month, month, 'month'))

        decadal_stats = (
            month_collection
            .map(lambda img: ee.Feature(None,img.rename('ET').reduceRegion(
                reducer=ee.Reducer.median(),
                geometry=ft.geometry(),
                scale=30
            )).set('date', img.get('system:time_start')))
        )

        #monthly sum of class_collection ET_blue
        monthly_sum = decadal_stats.aggregate_array('ET').reduce(ee.Reducer.sum())

        ft = ee.Feature(ft).set(f'ETA_0{month}', monthly_sum)

    return ee.Feature(ft)


In [ ]:

et_blue_collection=ee.FeatureCollection(output_asset_path)
print(f"ET blue collection size: {et_blue_collection.size().getInfo()}")

##theres a problem with duplicate features in assets. lets remove them
n = et_blue_collection.first().propertyNames()
n = n.remove('system:index')
# et_blue_collection = et_blue_collection.distinct(n)
print(f"ET blue collection size: {et_blue_collection.distinct(n).size().getInfo()}")

# Filter out unirrigated fields
et_blue_collection = et_blue_collection.filter(ee.Filter.neq('unirrigated', 1))

# average of residuals
median_abs_res_list = et_blue_collection.aggregate_array('residual').reduce(ee.Reducer.mean())
mean_abs_res_list = et_blue_collection.aggregate_array('residual2').reduce(ee.Reducer.mean())
# print("Average of 'median_abs_res':", median_abs_res_list.getInfo())
# print("Average of 'mean_abs_res':", mean_abs_res_list.getInfo())

#unique number of field_id
unique_field_ids = et_blue_collection.aggregate_array('field_id').distinct()

landuse_with_ETblue = landuse_collection.filter(ee.Filter.inList('field_id', unique_field_ids))
# Function to calculate stats for each field_id
def calculate_stats(ft):
    field_id = ee.String(ee.Feature(ft).get('field_id'))
    class_collection = et_blue_collection.filter(ee.Filter.eq('field_id', field_id))
    class_size = class_collection.size()
    class_id = ee.Number(class_collection.first().get('class'))
    # unique_fields = class_collection.aggregate_array('field_id').distinct().size()
    ETa_ETc = class_collection.aggregate_array('ETa_ETc').reduce(ee.Reducer.mean())
    ETa_ETc_max = class_collection.aggregate_array('ETa_ETc').reduce(ee.Reducer.max())
    ETa_ETc_min = class_collection.aggregate_array('ETa_ETc').reduce(ee.Reducer.min())
    avg_ETblue = class_collection.aggregate_array('ET_blue').reduce(ee.Reducer.sum())
    avg_residual = class_collection.aggregate_array('residual').reduce(ee.Reducer.mean())
    avg_area_per_field = ee.Number(class_collection.first().get('flaeche_m2'))
    avg_features_per_field = class_size

    ETblue_05 = class_collection.filter(ee.Filter.eq('month',5))
    ETblue_05 = ee.Algorithms.If(ETblue_05.size().eq(0), 0, ETblue_05.aggregate_array('ET_blue').reduce(ee.Reducer.sum()))
    ETblue_06 = class_collection.filter(ee.Filter.eq('month',6))
    ETblue_06 = ee.Algorithms.If(ETblue_06.size().eq(0), 0, ETblue_06.aggregate_array('ET_blue').reduce(ee.Reducer.sum()))
    ETblue_07 = class_collection.filter(ee.Filter.eq('month',7))
    ETblue_07 = ee.Algorithms.If(ETblue_07.size().eq(0), 0, ETblue_07.aggregate_array('ET_blue').reduce(ee.Reducer.sum()))
    ETblue_08 = class_collection.filter(ee.Filter.eq('month',8))
    ETblue_08 = ee.Algorithms.If(ETblue_08.size().eq(0), 0, ETblue_08.aggregate_array('ET_blue').reduce(ee.Reducer.sum()))
    ETblue_09 = class_collection.filter(ee.Filter.eq('month',9))
    ETblue_09 = ee.Algorithms.If(ETblue_09.size().eq(0), 0, ETblue_09.aggregate_array('ET_blue').reduce(ee.Reducer.sum()))

    total_ETblue = ee.Number(avg_ETblue).divide(1000).multiply(avg_area_per_field)
    return ee.Feature(ft).set({
        'field_id': field_id,
        'class': class_id,
        'total_features': class_size,
        # 'unique_fields': unique_fields,
        'avg_features_per_field': avg_features_per_field,
        'ETa_ETc_avg': ETa_ETc,
        'ETa_ETc_min': ETa_ETc_min,
        'ETa_ETc_max': ETa_ETc_max,
        'ETblue_mm': avg_ETblue,
        'ETblue_vol': total_ETblue,
        'ETblue_05': ETblue_05,
        'ETblue_06': ETblue_06,
        'ETblue_07': ETblue_07,
        'ETblue_08': ETblue_08,
        'ETblue_09': ETblue_09,
        # 'avg_area_per_field': avg_area_per_field,
        'avg_residual': avg_residual
    })

field_stats = landuse_with_ETblue.map(calculate_stats).map(calculate_stats_ET).set('ETa_ETc_threshold', ETa_ETc_threshold).set('ETblue_threshold', ETblue_threshold)


export_task = ee.batch.Export.table.toAsset(
    collection=field_stats,
    description=f"ETblue_field_stats_{CANTON}_{YEAR}",
    assetId=output_asset_path_stats
)
export_task.start()
print(f"Export task started: {output_asset_path_stats}")




ET blue collection size: 92536
ET blue collection size: 92536
Export task started: projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_annual_v2_withETA/ETblue_field_stats_Zuerich_2018_ETaETc70_ETblue10


In [498]:
# et_blue_collection=ee.FeatureCollection(output_asset_path)

# # # Focus on Bibertal
# #merge with Thurgau and Zuerich, because canton attribute is not accurate
# asset_path = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Zuerich_{YEAR}_ETaETc70_ETblue10"
# et_blue_collection = et_blue_collection.merge(ee.FeatureCollection(asset_path))
# asset_path = f"projects/thurgau-irrigation/assets/ZH_SH_TG/ET_blue_per_field_filtered_v2/ETblue_field_ts_Thurgau_{YEAR}_ETaETc70_ETblue10"
# et_blue_collection = et_blue_collection.merge(ee.FeatureCollection(asset_path))
# bibertal_boundary = ee.FeatureCollection('projects/thurgau-irrigation/assets/Schaffhausen/perimeter_bew_oberflaechengew')
# landuse_collection_bibertal=landuse_collection.filterBounds(bibertal_boundary.geometry())
# unique_field_ids = landuse_collection_bibertal.aggregate_array('field_id').distinct()
# et_blue_collection = et_blue_collection.filter(ee.Filter.inList('field_id', unique_field_ids))

# # Filter out unirrigated fields
# et_blue_collection = et_blue_collection.filter(ee.Filter.neq('unirrigated', 1))

# print(f"ET blue collection size after filtering: {et_blue_collection.size().getInfo()}")

# unique_field_ids = et_blue_collection.aggregate_array('field_id').distinct().size()

# print(f"Number of unique fields with significant ET_blue: {unique_field_ids.getInfo()}")
# # average features per field
# avg_features_per_field = et_blue_collection.size().divide(unique_field_ids)
# print(f"Average features per field: {avg_features_per_field.getInfo()}")
# # average features per field and class

# # Function to calculate stats for each class
# def calculate_class_stats(class_num):
#     class_collection = et_blue_collection.filter(ee.Filter.eq('class', class_num))
#     class_size = class_collection.size()
#     unique_fields = class_collection.aggregate_array('field_id').distinct().size()
#     ETa_ETc = class_collection.aggregate_array('ETa_ETc').reduce(ee.Reducer.median())
#     avg_ETblue = class_collection.aggregate_array('ET_blue').reduce(ee.Reducer.mean())
#     avg_residual = class_collection.aggregate_array('residual').reduce(ee.Reducer.mean())
#     avg_area_per_field = class_collection.aggregate_array('flaeche_m2').reduce(ee.Reducer.mean())
#     avg_features_per_field = class_size.divide(unique_fields)
#     # if class_size is 0, return zeros
#     avg_ETblue = ee.Algorithms.If(class_size.eq(0), 0, avg_ETblue)
#     avg_features_per_field = ee.Algorithms.If(class_size.eq(0), 0, avg_features_per_field)
#     avg_area_per_field = ee.Algorithms.If(class_size.eq(0), 0, avg_area_per_field)
#     avg_residual = ee.Algorithms.If(class_size.eq(0), 0, avg_residual)
#     ETa_ETc = ee.Algorithms.If(class_size.eq(0), 0, ETa_ETc)
#     # Total ETblue per class (field * features/field * Etblue/1000 * area)
#     total_ETblue = ee.Number(unique_fields).multiply(ee.Number(avg_ETblue).divide(1000)).multiply(avg_area_per_field).multiply(avg_features_per_field)
#     return ee.Feature(None, {
#         'class': class_num,
#         'total_features': class_size,
#         'unique_fields': unique_fields,
#         'avg_features_per_field': avg_features_per_field,
#         'avg_ETa_ETc': ETa_ETc,
#         'avg_ETblue': avg_ETblue,
#         'total_ETblue': total_ETblue,
#         'avg_area_per_field': avg_area_per_field,
#         'avg_residual': avg_residual
#     })

# # Map over all classes and calculate stats
# unique_classes = et_blue_collection.aggregate_array('class').distinct().sort()
# class_stats = unique_classes.map(calculate_class_stats)
# print('Total ET blue (m2):', ee.FeatureCollection(class_stats).aggregate_sum('total_ETblue').getInfo())


# # Print results for each class
# def print_class_stats(class_num):
#     stats = calculate_class_stats(class_num).getInfo()['properties']
#     print(f"Class {stats['class']}: {stats['total_features']} features, "
#           f"{stats['unique_fields']} fields, "
#           f"{stats['avg_features_per_field']:.2f} features/field, "
#           f"{stats['avg_ETa_ETc']:.2f} med ETa/ETc, "
#           f"{stats['avg_ETblue']:.2f} mean ET blue, "
#           f"{stats['avg_residual']:.2f} mean residual, "
#           f"{stats['avg_area_per_field']:.2f} mean area/field, "
#           f"{stats['total_ETblue']:.2f} total ET blue")

# # Print stats for each class
# for class_num in unique_classes.getInfo():
#     print_class_stats(class_num)


# # Function to calculate stats for every month and class
# def calculate_class_stats_per_month(class_num, month):
#     class_collection = et_blue_collection.filter(ee.Filter.eq('class', class_num)).filter(ee.Filter.eq('month', month))
#     class_size = class_collection.size()
#     unique_fields = class_collection.aggregate_array('field_id').distinct().size()
#     ETa_ETc = class_collection.aggregate_array('ETa_ETc').reduce(ee.Reducer.median())
#     avg_ETblue = class_collection.aggregate_array('ET_blue').reduce(ee.Reducer.mean())
#     avg_residual = class_collection.aggregate_array('residual').reduce(ee.Reducer.mean())
#     avg_area_per_field = class_collection.aggregate_array('flaeche_m2').reduce(ee.Reducer.mean())
#     avg_features_per_field = class_size.divide(unique_fields)
#     # if class_size is 0, return zeros
#     avg_ETblue = ee.Algorithms.If(class_size.eq(0), 0, avg_ETblue)
#     avg_features_per_field = ee.Algorithms.If(class_size.eq(0), 0, avg_features_per_field)
#     avg_area_per_field = ee.Algorithms.If(class_size.eq(0), 0, avg_area_per_field)
#     avg_residual = ee.Algorithms.If(class_size.eq(0), 0, avg_residual)
#     ETa_ETc = ee.Algorithms.If(class_size.eq(0), 0, ETa_ETc)
#     # Total ETblue per class (field * features/field * Etblue/1000 * area)
#     total_ETblue = ee.Number(unique_fields).multiply(ee.Number(avg_ETblue).divide(1000)).multiply(avg_area_per_field).multiply(avg_features_per_field)
#     return ee.Feature(None, {
#         'class': class_num,
#         'total_features': class_size,
#         'unique_fields': unique_fields,
#         'avg_features_per_field': avg_features_per_field,
#         'avg_ETa_ETc': ETa_ETc,
#         'avg_ETblue': avg_ETblue,
#         'total_ETblue': total_ETblue,
#         'avg_area_per_field': avg_area_per_field,
#         'avg_residual': avg_residual
#     })

# # Print results for each class
# def print_class_stats_per_month(class_num, month):
#     stats = calculate_class_stats_per_month(class_num, month).getInfo()['properties']
#     print(f"Month {month}, Class {class_num}: {stats['total_features']} features, "
#           f"{stats['unique_fields']} fields, "
#           f"{stats['avg_features_per_field']:.2f} features/field, "
#           f"{stats['avg_ETa_ETc']:.2f} med ETa/ETc, "
#           f"{stats['avg_ETblue']:.2f} mean ET blue, "
#           f"{stats['avg_residual']:.2f} mean residual, "
#           f"{stats['avg_area_per_field']:.2f} mean area/field, "
#           f"{stats['total_ETblue']:.2f} total ET blue")

# # Print stats for vegetables per month
# for month in ee.List.sequence(5, 9).getInfo():
#     print_class_stats_per_month(1, month)
#     print_class_stats_per_month(2, month)


# ##2018 results
# # ET blue collection size after filtering: 408
# # Number of unique fields with significant ET_blue: 101
# # Average features per field: 4.03960396039604
# # Total ET blue (m2): 68585.72566475355
# # Class 1: 215 features, 55 fields, 3.91 features/field, 0.50 med ETa/ETc, 10.81 mean ET blue, 6.53 mean residual, 16508.70 mean area/field, 38370.77 total ET blue
# # Class 2: 37 features, 11 fields, 3.36 features/field, 0.44 med ETa/ETc, 7.67 mean ET blue, 7.37 mean residual, 15286.60 mean area/field, 4335.54 total ET blue
# # Class 3: 61 features, 15 fields, 4.07 features/field, 0.50 med ETa/ETc, 7.22 mean ET blue, 6.39 mean residual, 17595.83 mean area/field, 7744.30 total ET blue
# # Class 4: 93 features, 19 fields, 4.89 features/field, 0.46 med ETa/ETc, 8.71 mean ET blue, 6.31 mean residual, 21457.54 mean area/field, 17388.99 total ET blue
# # Class 5: 2 features, 1 fields, 2.00 features/field, 0.52 med ETa/ETc, 12.01 mean ET blue, 7.01 mean residual, 31062.64 mean area/field, 746.12 total ET blue
# # Month 5, Class 1: 28 features, 18 fields, 1.56 features/field, 0.60 med ETa/ETc, 5.45 mean ET blue, 5.19 mean residual, 17547.81 mean area/field, 2679.92 total ET blue
# # Month 5, Class 2: 7 features, 4 fields, 1.75 features/field, 0.61 med ETa/ETc, 8.42 mean ET blue, 5.50 mean residual, 14331.88 mean area/field, 844.94 total ET blue
# # Month 6, Class 1: 48 features, 31 fields, 1.55 features/field, 0.60 med ETa/ETc, 10.83 mean ET blue, 5.47 mean residual, 15630.34 mean area/field, 8128.61 total ET blue
# # Month 6, Class 2: 4 features, 2 fields, 2.00 features/field, 0.58 med ETa/ETc, 4.80 mean ET blue, 4.56 mean residual, 15201.15 mean area/field, 291.70 total ET blue
# # Month 7, Class 1: 81 features, 45 fields, 1.80 features/field, 0.44 med ETa/ETc, 13.75 mean ET blue, 7.57 mean residual, 16272.48 mean area/field, 18117.59 total ET blue
# # Month 7, Class 2: 15 features, 9 fields, 1.67 features/field, 0.44 med ETa/ETc, 7.87 mean ET blue, 9.45 mean residual, 15836.71 mean area/field, 1869.31 total ET blue
# # Month 8, Class 1: 51 features, 33 fields, 1.55 features/field, 0.35 med ETa/ETc, 10.31 mean ET blue, 7.04 mean residual, 16815.32 mean area/field, 8838.14 total ET blue
# # Month 8, Class 2: 9 features, 6 fields, 1.50 features/field, 0.35 med ETa/ETc, 8.27 mean ET blue, 7.47 mean residual, 16493.13 mean area/field, 1227.69 total ET blue
# # Month 9, Class 1: 7 features, 7 fields, 1.00 features/field, 0.51 med ETa/ETc, 1.79 mean ET blue, 3.44 mean residual, 18874.61 mean area/field, 236.19 total ET blue
# # Month 9, Class 2: 2 features, 2 fields, 1.00 features/field, 0.51 med ETa/ETc, 6.50 mean ET blue, 3.44 mean residual, 9243.88 mean area/field, 120.17 total ET blue
# ##2020 results
# # ET blue collection size after filtering: 263
# # Number of unique fields with significant ET_blue: 72
# # Average features per field: 3.6527777777777777
# # Total ET blue (m2): 33827.86836366106
# # Class 1: 120 features, 34 fields, 3.53 features/field, 0.60 med ETa/ETc, 7.55 mean ET blue, 5.65 mean residual, 21307.62 mean area/field, 19303.80 total ET blue
# # Class 2: 34 features, 7 fields, 4.86 features/field, 0.63 med ETa/ETc, 5.52 mean ET blue, 4.75 mean residual, 17613.89 mean area/field, 3307.88 total ET blue
# # Class 3: 50 features, 13 fields, 3.85 features/field, 0.65 med ETa/ETc, 6.52 mean ET blue, 4.60 mean residual, 17658.53 mean area/field, 5755.43 total ET blue
# # Class 4: 19 features, 4 fields, 4.75 features/field, 0.64 med ETa/ETc, 5.32 mean ET blue, 4.71 mean residual, 15327.89 mean area/field, 1549.55 total ET blue
# # Class 5: 40 features, 14 fields, 2.86 features/field, 0.56 med ETa/ETc, 5.80 mean ET blue, 3.56 mean residual, 16854.65 mean area/field, 3911.20 total ET blue
# # Month 5, Class 1: 14 features, 12 fields, 1.17 features/field, 0.62 med ETa/ETc, 5.99 mean ET blue, 4.25 mean residual, 23093.28 mean area/field, 1936.35 total ET blue
# # Month 5, Class 2: 5 features, 3 fields, 1.67 features/field, 0.52 med ETa/ETc, 8.24 mean ET blue, 6.05 mean residual, 14699.35 mean area/field, 605.32 total ET blue
# # Month 6, Class 1: 35 features, 25 fields, 1.40 features/field, 0.60 med ETa/ETc, 7.28 mean ET blue, 5.76 mean residual, 21616.99 mean area/field, 5507.25 total ET blue
# # Month 6, Class 2: 11 features, 6 fields, 1.83 features/field, 0.66 med ETa/ETc, 7.15 mean ET blue, 4.71 mean residual, 13500.87 mean area/field, 1061.96 total ET blue
# # Month 7, Class 1: 41 features, 24 fields, 1.71 features/field, 0.59 med ETa/ETc, 9.98 mean ET blue, 7.26 mean residual, 19189.10 mean area/field, 7851.00 total ET blue
# # Month 7, Class 2: 4 features, 4 fields, 1.00 features/field, 0.56 med ETa/ETc, 3.26 mean ET blue, 6.12 mean residual, 21235.58 mean area/field, 276.91 total ET blue
# # Month 8, Class 1: 12 features, 12 fields, 1.00 features/field, 0.55 med ETa/ETc, 6.84 mean ET blue, 4.68 mean residual, 19947.99 mean area/field, 1637.85 total ET blue
# # Month 8, Class 2: 1 features, 1 fields, 1.00 features/field, 0.69 med ETa/ETc, 1.88 mean ET blue, 4.44 mean residual, 37955.81 mean area/field, 71.30 total ET blue
# # Month 9, Class 1: 18 features, 9 fields, 2.00 features/field, 0.49 med ETa/ETc, 4.23 mean ET blue, 3.48 mean residual, 25049.14 mean area/field, 1906.24 total ET blue
# # Month 9, Class 2: 13 features, 5 fields, 2.60 features/field, 0.62 med ETa/ETc, 4.08 mean ET blue, 3.90 mean residual, 19536.00 mean area/field, 1036.23 total ET blue

# ##2021 results
# # ET blue collection size after filtering: 27
# # Number of unique fields with significant ET_blue: 9
# # Average features per field: 3
# # Total ET blue (m2): 5240.034961882664
# # Class 1: 27 features, 9 fields, 3.00 features/field, 0.46 med ETa/ETc, 6.10 mean ET blue, 3.57 mean residual, 31806.81 mean area/field, 5240.03 total ET blue
# # Month 5, Class 1: 5 features, 5 fields, 1.00 features/field, 0.49 med ETa/ETc, 7.60 mean ET blue, 3.44 mean residual, 22212.71 mean area/field, 844.08 total ET blue
# # Month 5, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 6, Class 1: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 6, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 7, Class 1: 3 features, 3 fields, 1.00 features/field, 0.62 med ETa/ETc, 7.08 mean ET blue, 4.66 mean residual, 27000.06 mean area/field, 573.16 total ET blue
# # Month 7, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 8, Class 1: 6 features, 6 fields, 1.00 features/field, 0.47 med ETa/ETc, 5.17 mean ET blue, 3.44 mean residual, 32494.09 mean area/field, 1007.32 total ET blue
# # Month 8, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 9, Class 1: 13 features, 7 fields, 1.86 features/field, 0.46 med ETa/ETc, 5.73 mean ET blue, 3.44 mean residual, 36288.89 mean area/field, 2704.16 total ET blue
# # Month 9, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue

# ##2022 results
# # Number of unique fields with significant ET_blue: 93
# # Average features per field: 5.602150537634409
# # Class 1: 279 features, 46 fields, 6.07 features/field, 0.60 med ETa/ETc, 6.10 mean ET blue, 4.43 mean residual, 15900.24 mean area/field, 27064.99 total ET blue
# # Class 2: 101 features, 16 fields, 6.31 features/field, 0.60 med ETa/ETc, 6.60 mean ET blue, 4.68 mean residual, 22305.53 mean area/field, 14869.02 total ET blue
# # Class 3: 26 features, 9 fields, 2.89 features/field, 0.64 med ETa/ETc, 5.15 mean ET blue, 4.12 mean residual, 16111.18 mean area/field, 2155.68 total ET blue
# # Class 4: 97 features, 17 fields, 5.71 features/field, 0.62 med ETa/ETc, 4.12 mean ET blue, 4.11 mean residual, 21745.46 mean area/field, 8686.59 total ET blue
# # Class 5: 18 features, 5 fields, 3.60 features/field, 0.42 med ETa/ETc, 6.39 mean ET blue, 4.88 mean residual, 21165.68 mean area/field, 2433.08 total ET blue
# # Total ET blue (m2): 51610.46771956782
# # Month 5, Class 1: 17 features, 13 fields, 1.31 features/field, 0.54 med ETa/ETc, 4.40 mean ET blue, 3.41 mean residual, 16575.07 mean area/field, 1238.82 total ET blue
# # Month 5, Class 2: 4 features, 4 fields, 1.00 features/field, 0.61 med ETa/ETc, 6.31 mean ET blue, 4.77 mean residual, 32215.84 mean area/field, 813.59 total ET blue
# # Month 6, Class 1: 58 features, 33 fields, 1.76 features/field, 0.64 med ETa/ETc, 5.96 mean ET blue, 4.04 mean residual, 15299.20 mean area/field, 5291.84 total ET blue
# # Month 6, Class 2: 20 features, 13 fields, 1.54 features/field, 0.67 med ETa/ETc, 4.05 mean ET blue, 3.80 mean residual, 25165.26 mean area/field, 2038.39 total ET blue
# # Month 7, Class 1: 117 features, 39 fields, 3.00 features/field, 0.58 med ETa/ETc, 6.89 mean ET blue, 4.42 mean residual, 15455.87 mean area/field, 12454.90 total ET blue
# # Month 7, Class 2: 41 features, 15 fields, 2.73 features/field, 0.58 med ETa/ETc, 6.47 mean ET blue, 5.29 mean residual, 21253.53 mean area/field, 5636.59 total ET blue
# # Month 8, Class 1: 77 features, 33 fields, 2.33 features/field, 0.49 med ETa/ETc, 5.89 mean ET blue, 5.09 mean residual, 16804.42 mean area/field, 7624.57 total ET blue
# # Month 8, Class 2: 32 features, 11 fields, 2.91 features/field, 0.57 med ETa/ETc, 8.69 mean ET blue, 4.58 mean residual, 20163.24 mean area/field, 5608.30 total ET blue
# # Month 9, Class 1: 10 features, 8 fields, 1.25 features/field, 0.53 med ETa/ETc, 2.20 mean ET blue, 3.44 mean residual, 16476.05 mean area/field, 362.21 total ET blue
# # Month 9, Class 2: 4 features, 3 fields, 1.33 features/field, 0.66 med ETa/ETc, 4.25 mean ET blue, 3.44 mean residual, 26017.91 mean area/field, 442.30 total ET blue

# ##2023 results
# # ET blue collection size before joining: 60315
# # ET blue collection size before filtering: 4152
# # Number of unique fields: 342
# # Distribution of Classes: {'1': 810, '2': 1018, '3': 472, '4': 780, '5': 1072}
# # Number of unique fields per class: {1: 54, 2: 70, 3: 32, 4: 52, 5: 134}
# # Fraction of dates with data available: 0.6435452793834296
# # Fractions of dates available per month: {5: 0, 6: 0.9910979228486647, 7: 0.4435590969455511, 8: 0.7573696145124716, 9: 0.7573696145124716}
# # Property names after filtering: ['ETa_ETc', 'date', 'decade', 'secondEnd', 'year', 'bezugsjahr', 'median_abs_res', 'mad_res', 'firstStart', 'class', 'field_id', 'ETF_modeled', 'secondStart', 'ET_blue', 'isDoubleCropping', 'n', 'stddev_abs_res', 'flaeche_m2', 'zero_fraction', 'nutzung', 'firstEnd', 'month', 'mean_abs_res', 'days', 'canton', 'system:index']
# # ET blue collection size after filtering: 2014
# # Number of unique fields with no significant ET_blue: 327
# # Field IDs of class 1: ['00000000000000000875', '00000000000000000880', '00000000000000000987', '0000000000000000084d', '00000000000000000844', '00000000000000000854', '0000000000000000084f', '00000000000000000861', '00000000000000000847', '00000000000000000877', '0000000000000000098a', '00000000000000000851', '0000000000000000087b', '00000000000000000845', '00000000000000000862', '00000000000000000882', '00000000000000000846', '0000000000000000084a', '00000000000000000884', '00000000000000000868', '0000000000000000086e', '00000000000000000870', '0000000000000000098c', '00000000000000000883', '00000000000000000881', '00000000000000000874', '00000000000000000885', '0000000000000000087a', '0000000000000000098b', '00000000000000000860', '00000000000000000b11', '0000000000000000085c', '0000000000000000085d', '0000000000000000087f']
# # Fields of class 1, max monthly_ET_blue: [20.130000591278076, 33.015654935360025, 26.079999208450317, 28.06999957561493, 14.069999694824219, 12.03000020980835, 29.09999942779541, 27.09999966621399, 18.03000020980835, 26.09999978542328, 25.58054871372148, 25.070000052452087, 10.100000619888306, 12, 19.100000262260437, 18.000000476837158, 26.09999942779541, 14.999999046325684, 11.885098419937432, 11.000000476837158, 19, 17.99999976158142, 18.000000953674316, 20.999999046325684, 25.999999618530275, 18.100000619888306, 22.100000262260437, 12.099999904632568, 33.576863408088684, 18.069998741149902, 25, 22.130000591278076, 31.0600004196167, 21.100000381469727]
# # Field IDs of class 2: ['00000000000000000964', '0000000000000000093b', '000000000000000008df', '00000000000000000985', '0000000000000000094b', '0000000000000000091a', '0000000000000000097f', '0000000000000000093f', '0000000000000000093c', '00000000000000000954', '00000000000000000957', '000000000000000008ff', '00000000000000000945', '000000000000000008e1', '00000000000000000969', '000000000000000008e3', '0000000000000000096a', '000000000000000008f0', '000000000000000008f6', '0000000000000000095f', '00000000000000000950', '00000000000000000980', '00000000000000000958', '000000000000000008ed', '000000000000000008f5', '00000000000000000986', '00000000000000000949', '00000000000000000906', '0000000000000000096d', '0000000000000000097e', '000000000000000008ee']
# # Fields of class 2, max monthly_ET_blue: [22.3333326180776, 15.100000381469727, 25.00000023841858, 10.999998807907104, 21.99999976158142, 0, 22.010001749073695, 10.1478198360548, 20.130000591278076, 21.357672607720303, 22.130000591278076, 17.06999897956848, 23.799999396006267, 22.999999523162842, 19.24745123059142, 24.100000381469727, 18.000000476837158, 13.999999284744263, 24, 23.000000953674316, 29, 16.999999387531417, 33.999999046325684, 14.1100013256073, 18.000000953674316, 21.010001182556152, 17.070000529289246, 27.00582663394602, 15.294684643969179, 10.010000944137573, 12]
# # Average of 'median_abs_res': 5.332590176515367
# # Average of 'mean_abs_res': 6.31249538456976
# # Distribution of ET_blue bins: {'1': 65, '2': 48, '3': 23, '4': 47, '5': 28, '6': 49, '7': 36, '8': 20, '9': 122}


# # ET blue collection size after filtering: 438
# # Number of unique fields with significant ET_blue: 106
# # Average features per field: 4.132075471698113
# # Class 1: 137 features, 34 fields, 4.03 features/field, 0.09 min ETa/ETc, 7.01 mean ET blue, 4.60 mean residual, 20537.33 mean area/field, 19737.23 total ET blue
# # Class 2: 143 features, 31 fields, 4.61 features/field, 0.27 min ETa/ETc, 6.03 mean ET blue, 6.56 mean residual, 19530.19 mean area/field, 16827.55 total ET blue
# # Class 3: 34 features, 11 fields, 3.09 features/field, 0.37 min ETa/ETc, 7.23 mean ET blue, 5.26 mean residual, 14248.94 mean area/field, 3501.62 total ET blue
# # Class 4: 89 features, 19 fields, 4.68 features/field, 0.27 min ETa/ETc, 5.08 mean ET blue, 4.99 mean residual, 23487.34 mean area/field, 10628.05 total ET blue
# # Class 5: 35 features, 11 fields, 3.18 features/field, 0.27 min ETa/ETc, 6.26 mean ET blue, 4.14 mean residual, 16983.45 mean area/field, 3718.59 total ET blue
# # Month 5, Class 1: 0 features, 0 fields, 0.00 features/field, 0.00 min ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 5, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 min ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 6, Class 1: 49 features, 25 fields, 1.96 features/field, 0.26 min ETa/ETc, 5.71 mean ET blue, 5.20 mean residual, 18977.01 mean area/field, 5311.38 total ET blue
# # Month 6, Class 2: 61 features, 26 fields, 2.35 features/field, 0.43 min ETa/ETc, 6.44 mean ET blue, 6.62 mean residual, 18392.17 mean area/field, 7222.38 total ET blue
# # Month 7, Class 1: 23 features, 23 fields, 1.00 features/field, 0.46 min ETa/ETc, 7.51 mean ET blue, 4.44 mean residual, 21288.85 mean area/field, 3676.71 total ET blue
# # Month 7, Class 2: 19 features, 19 fields, 1.00 features/field, 0.55 min ETa/ETc, 5.90 mean ET blue, 6.88 mean residual, 21992.00 mean area/field, 2466.36 total ET blue
# # Month 8, Class 1: 43 features, 25 fields, 1.72 features/field, 0.09 min ETa/ETc, 10.47 mean ET blue, 4.61 mean residual, 21307.19 mean area/field, 9595.33 total ET blue
# # Month 8, Class 2: 48 features, 27 fields, 1.78 features/field, 0.27 min ETa/ETc, 6.64 mean ET blue, 7.32 mean residual, 20385.91 mean area/field, 6500.57 total ET blue
# # Month 9, Class 1: 22 features, 15 fields, 1.47 features/field, 0.45 min ETa/ETc, 2.64 mean ET blue, 3.44 mean residual, 21722.17 mean area/field, 1262.44 total ET blue
# # Month 9, Class 2: 15 features, 12 fields, 1.25 features/field, 0.43 min ETa/ETc, 2.53 mean ET blue, 3.44 mean residual, 18301.56 mean area/field, 693.74 total ET blue

# # 2024 results
# # ET blue collection size after filtering: 197
# # Number of unique fields with significant ET_blue: 54
# # Average features per field: 3.6481481481481484
# # Total ET blue (m2): 18505.583981860196
# # Class 1: 83 features, 20 fields, 4.15 features/field, 0.59 med ETa/ETc, 6.11 mean ET blue, 4.41 mean residual, 17893.72 mean area/field, 9073.29 total ET blue
# # Class 2: 12 features, 4 fields, 3.00 features/field, 0.62 med ETa/ETc, 5.13 mean ET blue, 4.31 mean residual, 11594.67 mean area/field, 714.07 total ET blue
# # Class 3: 22 features, 4 fields, 5.50 features/field, 0.64 med ETa/ETc, 5.18 mean ET blue, 4.11 mean residual, 12860.14 mean area/field, 1466.14 total ET blue
# # Class 4: 27 features, 6 fields, 4.50 features/field, 0.60 med ETa/ETc, 4.75 mean ET blue, 4.13 mean residual, 17584.26 mean area/field, 2256.75 total ET blue
# # Class 5: 53 features, 20 fields, 2.65 features/field, 0.48 med ETa/ETc, 5.65 mean ET blue, 4.28 mean residual, 16675.08 mean area/field, 4995.34 total ET blue
# # Month 5, Class 1: 1 features, 1 fields, 1.00 features/field, 0.68 med ETa/ETc, 5.00 mean ET blue, 3.44 mean residual, 19046.00 mean area/field, 95.23 total ET blue
# # Month 5, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 6, Class 1: 7 features, 7 fields, 1.00 features/field, 0.64 med ETa/ETc, 5.60 mean ET blue, 3.44 mean residual, 18032.57 mean area/field, 706.45 total ET blue
# # Month 6, Class 2: 2 features, 2 fields, 1.00 features/field, 0.67 med ETa/ETc, 3.50 mean ET blue, 4.44 mean residual, 12838.00 mean area/field, 89.87 total ET blue
# # Month 7, Class 1: 19 features, 13 fields, 1.46 features/field, 0.58 med ETa/ETc, 6.38 mean ET blue, 4.07 mean residual, 18132.32 mean area/field, 2199.23 total ET blue
# # Month 7, Class 2: 0 features, 0 fields, 0.00 features/field, 0.00 med ETa/ETc, 0.00 mean ET blue, 0.00 mean residual, 0.00 mean area/field, 0.00 total ET blue
# # Month 8, Class 1: 32 features, 16 fields, 2.00 features/field, 0.62 med ETa/ETc, 5.80 mean ET blue, 5.18 mean residual, 18509.16 mean area/field, 3434.81 total ET blue
# # Month 8, Class 2: 8 features, 3 fields, 2.67 features/field, 0.61 med ETa/ETc, 5.80 mean ET blue, 4.46 mean residual, 12542.50 mean area/field, 582.10 total ET blue
# # Month 9, Class 1: 24 features, 13 fields, 1.85 features/field, 0.46 med ETa/ETc, 6.50 mean ET blue, 3.98 mean residual, 16795.75 mean area/field, 2620.60 total ET blue
# # Month 9, Class 2: 2 features, 1 fields, 2.00 features/field, 0.61 med ETa/ETc, 4.09 mean ET blue, 3.54 mean residual, 6560.00 mean area/field, 53.63 total ET blue

